## Install app requirements

In [ ]:
!pip install --upgrade pip
!pip install -r requirements.txt
!source .venv/bin/activate

___

## Connect to Spark Cluster using SparkConnect

### Load environment variables

In [7]:
from commons.constants import *
from analytics.sc_session import get_session

session = get_session()
session

In [8]:
from analytics.sc_data import load_collection
products_df = load_collection(session, MDB_PRODUCTS_COLLECTION)
print(f"""
    Hay {products_df.count()} productos en la colección {MDB_PRODUCTS_COLLECTION}
""")


    Hay 2040 productos en la colección mbdtfm_magento_catalog_products_mview



In [2]:
from analytics.sc_embeddings import create_product_embeddings_w2v
create_product_embeddings_w2v(session)

Successfully wrote embeddings to MongoDB into mbdtfm_magento_catalog_products_embeddings


In [9]:
embeds_df = load_collection(session, MDB_EMBEDDINGS_COLLECTION)
print(f"""
    Hay {embeds_df.count()} products en la colección {MDB_EMBEDDINGS_COLLECTION}
""")


    Hay 2040 products en la colección mbdtfm_magento_catalog_products_embeddings



In [ ]:
session.stop()